Here is a w2v model trained over the two jigsaw toxic comment competitions. As you can see I've not included the test data from the lastest competition - as you can see it doesn't include the test data from the lastest comp. Feel free to fork and add it if you need

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['jigsaw-unintended-bias-in-toxicity-classification', 'cleaned-toxic-comments']


Note (again) - deliberately don't use test2 here

In [2]:
df_train_1 = pd.read_csv('../input/cleaned-toxic-comments/train_preprocessed.csv')
df_test_1 = pd.read_csv('../input/cleaned-toxic-comments/test_preprocessed.csv')
df_train_2 = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
# df_test_2 = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

In [3]:
df_train_1.sample(3)

,comment_text,id,identity_hate,insult,obscene,set,severe_toxic,threat,toxic,toxicity
31668,freeing myself from bothering with being on th...,541b00ccfef15003,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
106911,hell my suspicion was true wikipedia is driven...,3bb2d4ae8fdccb12,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0
142875,also the canoe slam article is from years ago ...,fc14fc5a388ca1fb,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0


In [4]:
def basic_preprocess(data):
    data = str(data)
    data = data.lower()
    '''
    Credit goes to https://www.kaggle.com/gpreda/jigsaw-fast-compact-solution
    '''
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
    def clean_special_chars(text, punct):        
        for p in punct:
            text = text.replace(p, ' ')
        return text

    data = clean_special_chars(data, punct)
    return data

In [5]:
df_train_1['clean'] = df_train_1['comment_text'].apply(basic_preprocess)
df_test_1['clean'] = df_test_1['comment_text'].apply(basic_preprocess)
df_train_2['clean'] = df_train_2['comment_text'].apply(basic_preprocess)
# df_test_2['clean'] = df_test_2['comment_text'].apply(basic_preprocess)

To train gensim we need to prep the data. This is a 3 part process

- concatenate the data into a single list
- split each string into a list of words
- filter out some junk

At that point we have a list of lists

In [10]:
all_text = list(df_train_2['clean'] + df_train_1['clean'] + df_test_1['clean'])

all_text = list(map(lambda s: str(s).split(' '), all_text))
all_text = list(map(lambda s: list(filter(lambda w: str(w).strip() != "", s)), all_text))

all_text[:3]

[['this',
  'is',
  'so',
  'cool',
  'it',
  's',
  'like',
  'would',
  'you',
  'want',
  'your',
  'mother',
  'to',
  'read',
  'this',
  'really',
  'great',
  'idea',
  'well',
  'done',
  'explanation',
  'why',
  'the',
  'edits',
  'made',
  'under',
  'my',
  'username',
  'hardcore',
  'metallica',
  'fan',
  'were',
  'reverted',
  'they',
  'weren',
  't',
  'vandalisms',
  'just',
  'closure',
  'on',
  'some',
  'gas',
  'after',
  'i',
  'voted',
  'at',
  'new',
  'york',
  'dolls',
  'fac',
  'and',
  'please',
  'don',
  't',
  'remove',
  'the',
  'template',
  'from',
  'the',
  'talk',
  'page',
  'since',
  'i',
  'm',
  'retired',
  'now',
  'yo',
  'bitch',
  'ja',
  'rule',
  'is',
  'more',
  'succesful',
  'then',
  'you',
  'll',
  'ever',
  'be',
  'whats',
  'up',
  'with',
  'you',
  'and',
  'hating',
  'you',
  'sad',
  'mo',
  'fuck',
  'as',
  'i',
  'should',
  'bitch',
  'slap',
  'ur',
  'pethedic',
  'white',
  'faces',
  'and',
  'get',
  'you'

In [11]:
import gensim

We need to be careful with the default parameters of gensim. We want a vector size of 300 dimensions, a min occurance count of 1 and a window size (for training) of 5

In [ ]:
%%time

gensim_model = gensim.models.Word2Vec(all_text, size=300, min_count=1, iter=25, window=5)

In [ ]:
gensim_model.wv.most_similar("khadr")

In [ ]:
gensim_model.save('jigsaw_w2v_model')